# GCPy: Google Cloud ♥ Python 

A Python package to easily interface with Google Cloud Platform.

## Instructions for installation
1. Install gcloud SDK from [Link](https://cloud.google.com/sdk/docs/install)
2. Initialize gloud SDK: `gcloud init`
3. Authenticate your account: `gcloud auth application-default login`
4. Clone this repo: `git clone https://github.com/stephenleo/gcpy.git`
5. CD into the directory: `cd gcpy`
6. Install: `python setup.py install`

## Usage
Import the package

In [1]:
import gcpy

# query_to_gcs

Runs an SQL query on a Big Query table and stores the result into sharded csv files on a GCS bucket.

**Important!** Remember to create the GCS bucket you want to use before running

**Arguments:**
- `sql_query_file`: Full path to the .sql file that contains the query. 

    Any query parameters should be within {}. eg: `WHERE col_name<{some_key}`. See query_params argument.
    
- `target_gcs_path`: GCS bucket path to store the output of the query in sharded csvs

- `query_params`: Query Parameters dictionary. Default: `{}`

    The sql query in .sql is treated as a string with parameters inside {key} replaced their corresponding values. 
    
    eg: `{'some_key':10}` will change the sql: `WHERE col_name<{some_key}` --> `WHERE col_name<10`
    
- `project`: GCP Project name. Default: `CLIENT.project`

- `dataset`: Big Query Dataset to temporarily store the results before moving to GCS. Default: `'gcpy'`

    It is the string that appears inbetween the project name and table name in Big Query.
        
- `del_temp_bq_table`: Switch to delete the temporary BQ table. Default: `True`. 

    If set to `False`, the query results are saved in BQ table `f'{project}.{dataset}.{sql_query_file.split("/")[-1].split(".")[0]}_{current_date_time}'`

- `location`: location of the GCS bucket. Default: `'US'`

- `client`: The BigQuery Client. Default: `CLIENT`

**Returns:**
The string path to the stored sharded csvs

## Example 1: Without `query_params`

In [2]:
%%bash
# Write your SQL query into a .sql file.
echo """SELECT
  DISTINCT name,
  gender
FROM
  bigquery-public-data.usa_names.usa_1910_current""" > name_gender.sql

In [3]:
# Run your SQL query and save the results to your GCS bucket
result_path = gcpy.query_to_gcs(sql_query_file='name_gender.sql', 
                                target_gcs_path='gs://gcpy-bucket/us_name_gender')

print(f'Query results saved in: {result_path}')

2021-02-07 23:18:26.729 INFO:	Executing query
2021-02-07 23:18:30.079 INFO:	Query Execution time: 3.3476343154907227
2021-02-07 23:18:30.080 INFO:	Write to Sharded CSVs in GCS
2021-02-07 23:18:32.232 INFO:	Exported leo-gcp-sanbox.gcpy.name_gender_20210207231826 to gs://gcpy-bucket/us_name_gender/20210207231826/name_gender_*.csv
2021-02-07 23:18:32.233 INFO:	Time elapsed: 2.1513211727142334 seconds
2021-02-07 23:18:32.554 INFO:	Deleted leo-gcp-sanbox.gcpy.name_gender_20210207231826 temporary BQ Table


Query results saved in: gs://gcpy-bucket/us_name_gender/20210207231826/name_gender_*.csv


## Example 2: With `query_params`
`query_params` argument can be used to control the SQL query at runtime.

The below code will query all the names that start with `'A'`

In [4]:
%%bash
# Write your SQL query into a .sql file.
echo """SELECT
  DISTINCT name,
  gender
FROM
  bigquery-public-data.usa_names.usa_1910_current
WHERE
  name LIKE '{start_letter}%'""" > name_gender_start_letter.sql

In [5]:
# Run your SQL query and save the results to your GCS bucket
result_path = gcpy.query_to_gcs(sql_query_file='name_gender_start_letter.sql', 
                                target_gcs_path='gs://gcpy-bucket/us_name_gender', 
                                query_params = {'start_letter': 'A'})

print(f'Query results saved in: {result_path}')

2021-02-07 23:18:35.717 INFO:	Executing query
2021-02-07 23:18:38.156 INFO:	Query Execution time: 2.4370853900909424
2021-02-07 23:18:38.156 INFO:	Write to Sharded CSVs in GCS
2021-02-07 23:18:40.831 INFO:	Exported leo-gcp-sanbox.gcpy.name_gender_start_letter_20210207231835 to gs://gcpy-bucket/us_name_gender/20210207231835/name_gender_start_letter_*.csv
2021-02-07 23:18:40.832 INFO:	Time elapsed: 2.674234628677368 seconds
2021-02-07 23:18:41.130 INFO:	Deleted leo-gcp-sanbox.gcpy.name_gender_start_letter_20210207231835 temporary BQ Table


Query results saved in: gs://gcpy-bucket/us_name_gender/20210207231835/name_gender_start_letter_*.csv


# sharded_gcs_csv_to_pd

Loads sharded csv files on a GCS bucket into a single Pandas dataframe.

**Arguments:**
- `source_gcs_path`: source GCS path containing the sharded csv files to load to BQ

- `file_prefix`: A file name prefix to select only the files of interest


**Returns:**
A single pandas dataframe that concatenates all the sharded csvs

## Example
Load the sharded csv files that were queried in the `query_to_gcs` example into a dataframe `df`

In [6]:
df = gcpy.sharded_gcs_csv_to_pd(source_gcs_path='gs://gcpy-bucket/us_name_gender/20210207231835/', 
                                file_prefix='name_gender_')

print(df.shape)
df.head()

100%|██████████| 1/1 [00:01<00:00,  1.64s/it]

(3804, 2)


,name,gender
0,Alma,F
1,Antoinette,F
2,Anica,F
3,Ayah,F
4,Anaya,F


# pd_to_bq

Load a Pandas Dataframe into a BigQuery table
    
**Arguments:**
- `source_df`: The DataFrame to load into BQ

- `target_dataset`: BQ dataset containing the `target_tablename` table to import into

- `target_tablename`: BQ table to import into

- `project`: GCP Project name. Default: `CLIENT.project`

- `client`: The BigQuery Client. Default: `CLIENT`

**Returns:**
The string BQ table name where source_df has been uploaded to

## Example
Load the pandas dataframe `df` into a BQ table `f'{project}.{target_dataset}.{target_tablename}'`

In [7]:
result_table = gcpy.pd_to_bq(source_df=df, target_dataset='gcpy', target_tablename='us_name_gender')

print(f'Source Dataframe uploaded to BQ table: {result_table}')

2021-02-07 23:19:05.687 INFO:	<google.cloud.bigquery.job.load.LoadJob object at 0x7f96b5944730>


Source Dataframe uploaded to BQ table: leo-gcp-sanbox.gcpy.us_name_gender


# gcs_to_bq

Load sharded csvs from GCS into BQ

**Arguments:**
- `source_gcs_path`: source GCS path containing the files to load to BQ

- `target_bq_dataset`: BQ dataset containing the `"target_bq_tablename"` table to import into

- `target_bq_tablename`: BQ table to import into

- `target_table_schema`: `'auto'` or dictionary of the form `{'col_name': 'dtype'}`. Default: `'auto'`

- `num_header_rows`: Number of header rows to skip while data upload. Default: `1` will skip the first row (column header)

- `project`: GCP Project name. Default: `CLIENT.project`

- `client`: The BigQuery Client. Default: `CLIENT`

**Returns:**
The string BQ table name where csv files from source_gcs_path has been uploaded to

In [9]:
result_table = gcpy.gcs_to_bq(source_gcs_path='gs://gcpy-bucket/us_name_gender/20210207231835/*.csv', 
                              target_bq_dataset='gcpy', target_bq_tablename='us_name_gender_2', 
                              target_table_schema = {"name": "string", "gender": "string"})

print(f'CSV files in source GCS path uploaded to BQ table: {result_table}')

2021-02-07 23:19:59.517 INFO:	Starting job ceb45ecb-1fa5-4614-964d-522540a48b74
2021-02-07 23:20:02.004 INFO:	Job finished. Time elapsed: 2.944720506668091 seconds
2021-02-07 23:20:02.297 INFO:	Loaded 3804 rows


CSV files in source GCS path uploaded to BQ table: leo-gcp-sanbox.gcpy.us_name_gender_2
